In [1]:
import numpy as np

import cv2

import keras, keras.layers as L
import tensorflow as tf

import glob
import ntpath

from matplotlib import pyplot as plt

Using TensorFlow backend.


In [2]:
from collections import Counter

In [3]:
%matplotlib notebook

In [33]:
config = tf.ConfigProto( device_count = {'GPU': 0 , 'CPU': 8} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [5]:
path = r'C:/Users/mkotyushev/Documents/z/'
apparel_type = r"outwear/"

In [6]:
# def read_apparel(path, kp_number=128):
#     image_1_len = 840
#     image_0_len = 560
#     features = []
#     names = []

#     for filename in glob.glob(path + "*"):
#     #     print(ntpath.basename(filename).split("_")[0])
#         img1 = cv2.imread(filename, 0)
#         kaze = cv2.KAZE_create()
#         kp1, des1 = kaze.detectAndCompute(img1, None)

#         single_kp_features = []
#         for i in range(len(kp1)):
#                 new1 =  [
#                     kp1[i].response*10,
#                         kp1[i].pt[0]/image_0_len,
#                         kp1[i].pt[1]/image_1_len,
#                         kp1[i].size/image_0_len,
#                         kp1[i].angle/360,
#                         kp1[i].octave/3, 
#                         kp1[i].class_id/14,

#                 ] + des1[i].tolist() # + [int(ntpath.basename(filename).split("_")[0])]
#                 single_kp_features.append(new1)
#         print(filename)
#         features.append(
#             np.resize(
#                     np.array(
#                         single_kp_features)[
#                         np.array(
#                             single_kp_features)
#                         [:,0]
#                         .argsort()
#                  ],(kp_number, 71)
#             )
#         )
#         names.append(int(ntpath.basename(filename).split("_")[0]))
#     return features, names

In [7]:
def read_apparel(path, kp_number=64):
    image_1_len = 840
    image_0_len = 560
    features = []
    names = []
    kaze = cv2.KAZE_create()

    for filename in glob.glob(path + "*"):
    #     print(ntpath.basename(filename).split("_")[0])
        img1 = cv2.imread(filename, 0)
        
        kp1, des1 = kaze.detectAndCompute(img1, None)
        if len(kp1): 
            single_kp_features = []
            for i in range(len(kp1)):
                    new1 =  [
                        kp1[i].response*10,
                            kp1[i].pt[0]/image_0_len,
                            kp1[i].pt[1]/image_1_len,
                            kp1[i].size/image_0_len,
                            kp1[i].angle/360,
                            kp1[i].octave/3, 
                            kp1[i].class_id/14,

                    ] + des1[i].tolist() # + [int(ntpath.basename(filename).split("_")[0])]
                    single_kp_features.append(new1)


            features.append(
                np.resize(
                    sorted(single_kp_features, key = lambda x:x[0])
                    ,(kp_number, 71)
                )
            )
            names.append(int(ntpath.basename(filename).split("_")[0]))
    return features, names

In [44]:
path + "*.jpg"

'C:/Users/mkotyushev/Documents/z/*.jpg'

In [59]:
def create_kp(path, kp_number=2**13):
    image_1_len = 840
    image_0_len = 560
    features = []
    names = []
    kaze = cv2.KAZE_create()

    for filename in glob.glob(path + "**.jpg", recursive=True):
#         print(ntpath.basename(filename).split("_")[0])
        img1 = cv2.imread(filename, 0)
        kp1, des1 = kaze.detectAndCompute(img1, None)
        
        if len(kp1): 
            single_kp_features = []
            for i in range(len(kp1)):
                    new1 =  [
                        kp1[i].response*10,
                            kp1[i].pt[0]/image_0_len,
                            kp1[i].pt[1]/image_1_len,
                            kp1[i].size/image_0_len,
                            kp1[i].angle/360,
                            kp1[i].octave/3, 
                            kp1[i].class_id/14,

                    ] + des1[i].tolist()
                    single_kp_features.append(new1)

            np.save( (filename + "_kp.npy"),
                    sorted(single_kp_features, key = lambda x:x[0]))

In [62]:
create_kp(r'C:/Users/mkotyushev/Documents/z_tiny_cut/*')

In [8]:
def generate_data(path):
    X_data = []
    y_data = []
    
    features, names = read_apparel(path)

    for app_a, name_a in zip(features, names):
        for app_b, name_b in zip(features, names):
            X_data.append([app_a.tolist(), app_b.tolist()])
            y_data.append(float(name_a == name_b))
    return np.array(X_data), np.array(y_data)

In [9]:
X_train, y_train = generate_data(r'C:/Users/mkotyushev/Documents/z_tiny/')

In [10]:
X_train.shape

(37636, 2, 64, 71)

In [11]:
X_test, y_test = generate_data(r'C:/Users/mkotyushev/Documents/z_tiny_b/')

In [12]:
X_test.shape

(18225, 2, 64, 71)

In [39]:
for i in range(71):
    print("{} range: {} - {} \n".format(i, np.min(X_train[:,:,:,i]), np.max(X_train[:,:,:,i])))

0 range: 0.010000067995861173 - 0.6028455123305321 

1 range: 0.003026960790157318 - 0.9950328281947545 

2 range: 0.0018085760729653494 - 0.9969565981910342 

3 range: 0.00571729668549129 - 0.07083385331290108 

4 range: 4.639754382272561e-05 - 0.9999781290690104 

5 range: 0.0 - 1.0 

6 range: 0.07142857142857142 - 1.0 

7 range: -0.3947840631008148 - 0.44129785895347595 

8 range: -0.48490166664123535 - 0.5374051332473755 

9 range: 0.0 - 0.4587584435939789 

10 range: 0.0 - 0.5634503364562988 

11 range: -0.336716890335083 - 0.3110734820365906 

12 range: -0.47247153520584106 - 0.40484973788261414 

13 range: 0.0 - 0.5884550213813782 

14 range: 0.0 - 0.5732281804084778 

15 range: -0.39618349075317383 - 0.3488147556781769 

16 range: -0.5078930258750916 - 0.661106288433075 

17 range: 1.0938735334198313e-10 - 0.6280149817466736 

18 range: 1.6665574475993594e-10 - 0.7025898694992065 

19 range: -0.4181159436702728 - 0.36405831575393677 

20 range: -0.38319459557533264 - 0.62703436

In [13]:
model = keras.models.Sequential()
model.add(L.InputLayer(input_shape=[2, 64, 71]))
model.add(L.BatchNormalization())
model.add(L.Flatten())
model.add(L.Dense(512, activation='relu'))
model.add(L.Dense(256, activation='relu'))
model.add(L.Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [14]:
model.fit(np.array(X_train), (np.array(y_train)), epochs=10, validation_data=[(np.array(X_test)), (np.array(y_test))])

Train on 37636 samples, validate on 18225 samples
Epoch 1/10
37636/37636 [==============================] - 21s 571us/step - loss: 0.1867 - val_loss: 0.0660
Epoch 2/10
37636/37636 [==============================] - 18s 482us/step - loss: 0.0289 - val_loss: 0.0657
Epoch 3/10
37636/37636 [==============================] - 18s 479us/step - loss: 0.0194 - val_loss: 0.0671
Epoch 4/10
37636/37636 [==============================] - 18s 474us/step - loss: 0.0136 - val_loss: 0.0667
Epoch 5/10
37636/37636 [==============================] - 18s 478us/step - loss: 0.0118 - val_loss: 0.0664
Epoch 6/10
37636/37636 [==============================] - 18s 488us/step - loss: 0.0124 - val_loss: 0.0662
Epoch 7/10
37636/37636 [==============================] - 19s 492us/step - loss: 0.0118 - val_loss: 0.0661
Epoch 8/10
37636/37636 [==============================] - 18s 484us/step - loss: 0.0099 - val_loss: 0.0656
Epoch 9/10
37636/37636 [==============================] - 18s 482us/step - loss: 0.0094 - val_

In [15]:
model.fit(np.array(X_train), (np.array(y_train)), epochs=100, validation_data=[(np.array(X_test)), (np.array(y_test))])

Train on 37636 samples, validate on 18225 samples
Epoch 1/100
37636/37636 [==============================] - 18s 485us/step - loss: 0.0073 - val_loss: 0.0648
Epoch 2/100
37636/37636 [==============================] - 18s 479us/step - loss: 0.0062 - val_loss: 0.0651
Epoch 3/100
37636/37636 [==============================] - 18s 474us/step - loss: 0.0054 - val_loss: 0.0651
Epoch 4/100
37636/37636 [==============================] - 18s 474us/step - loss: 0.0044 - val_loss: 0.0656
Epoch 5/100
37636/37636 [==============================] - 18s 477us/step - loss: 0.0034 - val_loss: 0.0681
Epoch 6/100
37636/37636 [==============================] - 18s 476us/step - loss: 0.0032 - val_loss: 0.0719
Epoch 7/100
37636/37636 [==============================] - 18s 476us/step - loss: 0.0022 - val_loss: 0.0724
Epoch 8/100
37636/37636 [==============================] - 18s 475us/step - loss: 0.0021 - val_loss: 0.0761
Epoch 9/100
18432/37636 [=============>................] - ETA: 8s - loss: 0.0020

KeyboardInterrupt: 

In [20]:
model = keras.models.Sequential()
model.add(L.InputLayer(input_shape=[2, 64, 71]))
model.add(L.BatchNormalization())
model.add(L.Flatten())
model.add(L.Dense(32, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.1))
model.add(L.Dense(32, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.1))
model.add(L.Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [34]:
model.fit(np.array(X_train), (np.array(y_train)), epochs=10, validation_data=[(np.array(X_test)), (np.array(y_test))])

Train on 37636 samples, validate on 18225 samples
Epoch 1/10
37636/37636 [==============================] - 34s 914us/step - loss: 0.2122 - val_loss: 0.0656
Epoch 2/10
37636/37636 [==============================] - 32s 848us/step - loss: 0.0496 - val_loss: 0.0655
Epoch 3/10
37636/37636 [==============================] - 32s 848us/step - loss: 0.0427 - val_loss: 0.0660
Epoch 4/10
37636/37636 [==============================] - 33s 864us/step - loss: 0.0382 - val_loss: 0.0657
Epoch 5/10
37636/37636 [==============================] - 32s 853us/step - loss: 0.0351 - val_loss: 0.0654
Epoch 6/10
37636/37636 [==============================] - 32s 845us/step - loss: 0.0334 - val_loss: 0.0644
Epoch 7/10
37636/37636 [==============================] - 32s 857us/step - loss: 0.0314 - val_loss: 0.0644
Epoch 8/10
37636/37636 [==============================] - 32s 857us/step - loss: 0.0313 - val_loss: 0.0640
Epoch 9/10
37636/37636 [==============================] - 32s 839us/step - loss: 0.0301 - val_

In [35]:
model.fit(np.array(X_train), (np.array(y_train)), epochs=10, validation_data=[(np.array(X_test)), (np.array(y_test))])

Train on 37636 samples, validate on 18225 samples
Epoch 1/10
37636/37636 [==============================] - 33s 872us/step - loss: 0.0266 - val_loss: 0.0652
Epoch 2/10
37636/37636 [==============================] - 32s 856us/step - loss: 0.0253 - val_loss: 0.0674
Epoch 3/10
37636/37636 [==============================] - 32s 863us/step - loss: 0.0234 - val_loss: 0.0795
Epoch 4/10
37636/37636 [==============================] - 32s 857us/step - loss: 0.0212 - val_loss: 0.0798
Epoch 5/10
37636/37636 [==============================] - 32s 858us/step - loss: 0.0194 - val_loss: 0.0843
Epoch 6/10
37636/37636 [==============================] - 32s 857us/step - loss: 0.0180 - val_loss: 0.1325
Epoch 7/10
37636/37636 [==============================] - 33s 870us/step - loss: 0.0166 - val_loss: 0.1126
Epoch 8/10
37636/37636 [==============================] - 33s 874us/step - loss: 0.0155 - val_loss: 0.1003
Epoch 9/10
37636/37636 [==============================] - 33s 867us/step - loss: 0.0145 - val_

In [15]:
model.fit(np.array(X_train), (np.array(y_train)), epochs=100, validation_data=[(np.array(X_test)), (np.array(y_test))])

Train on 37636 samples, validate on 18225 samples
Epoch 1/100
37636/37636 [==============================] - 18s 485us/step - loss: 0.0073 - val_loss: 0.0648
Epoch 2/100
37636/37636 [==============================] - 18s 479us/step - loss: 0.0062 - val_loss: 0.0651
Epoch 3/100
37636/37636 [==============================] - 18s 474us/step - loss: 0.0054 - val_loss: 0.0651
Epoch 4/100
37636/37636 [==============================] - 18s 474us/step - loss: 0.0044 - val_loss: 0.0656
Epoch 5/100
37636/37636 [==============================] - 18s 477us/step - loss: 0.0034 - val_loss: 0.0681
Epoch 6/100
37636/37636 [==============================] - 18s 476us/step - loss: 0.0032 - val_loss: 0.0719
Epoch 7/100
37636/37636 [==============================] - 18s 476us/step - loss: 0.0022 - val_loss: 0.0724
Epoch 8/100
37636/37636 [==============================] - 18s 475us/step - loss: 0.0021 - val_loss: 0.0761
Epoch 9/100
18432/37636 [=============>................] - ETA: 8s - loss: 0.0020

KeyboardInterrupt: 